# @ 주제 : 한국인 메이저리거의 현지 평가 분석

# 1. 가설
* Naive Bayes 분류기를 사용하여 현재 한국인 메이저리거의 현지 평가가 긍정적인지 부정적인지 분석할 수 있다.

# 2. 연구방법
## 2.1 데이터 수집
* twitter에서 영문으로 작성된 메이저리거 관련 글을 수집하기로 함
* 하지만, 한국인 메이저리거 관련 트윗의 수가 매우 적고 그 중 정서가 드러난 글은 더더욱 부족함
* 한국인 메이저리거 뿐만 아닌 전체 메이저리거 선수 관련 트윗을 검색하여 정서가 드러난 300 document를 추출함
* 해당 document가 긍정적인 정서인 경우 1, 부정적인 정서인 경우 0으로 class를 분류함

> Code - twitter API를 이용한 트위터 해쉬태그 검색

In [2]:
import twitter

# twitter API 사용을 위한 4가지 토큰
consumer_key="R8a6YSb4yf77X27emnGcI9hKdYYAEDbSqK08ocg6s5Vz9yLnr6"
consumer_secret="NIM6wPYiI2S4rCRkAPZUpaCpa"
token_key="VKTOHImjOvE4Kp8ECj3T11zzorHuwxUImNnfWGuuVIaK9"
token_secret="2373074766-PL14cizo2IrNPcvcUI25kdi7KjZQMAdRnNx2SPN"

api = twitter.Twitter(auth=twitter.OAuth(token_secret, token_key,consumer_secret, consumer_key))
    
response=api.search.tweets(q="#park")
str = response['statuses']
print(str)

[{u'contributors': None, u'truncated': False, u'text': u'Sleeping giant from Joeboy Lake \n#Trail #Park #SleepingGiant #Sky #Lake #Clouds #MakeItCount\u2026 https://t.co/9Z28BNFSVD', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 743627969172692992L, u'favorite_count': 0, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [33, 39], u'text': u'Trail'}, {u'indices': [40, 45], u'text': u'Park'}, {u'indices': [46, 60], u'text': u'SleepingGiant'}, {u'indices': [61, 65], u'text': u'Sky'}, {u'indices': [66, 71], u'text': u'Lake'}, {u'indices': [72, 79], u'text': u'Clouds'}, {u'indices': [80, 92], u'text': u'MakeItCount'}], u'urls': [{u'url': u'https://t.co/9Z28BNFSVD', u'indices': [94, 117], u'expanded_url': u'https://www.instagram.com/p/BGvQteBsl0p/', u'display_url': u'instagram.com/p/BGvQteBsl0p/'}]}, u'retweeted': False, u'coordinates': None, u'source': u'<a href="http://instagram.com" rel="nofollow">Instagram</a>', u'in_reply_to_screen_name':

## 2.2 데이터 처리
* 수집한 document들을 단어 벡터로 변환하는 작업이 필요함
* 단어 벡터 중에서 관사나 이름 명사 등 정서와 관련없는 단어(stop word)를 필터링함
* 이 때, 해쉬태그를 포함한 단어도 정서를 나타내는 경우가 있어 이를 특수문자가 제거된 순수단어 벡터 형태로 변환하는 작업이 선행됨


> Code - text parsing(특수 문자 및 길이가 1인 word 제거)

In [83]:
import numpy as np
import re

docArr = []
classArr = []
regEx = re.compile(r'\W*') #complie: 정규식을 두 번 이상 사용할 경우 효율을 위함. '\W*': a-z, A-Z, 0-9 문자열

file = open('data/trainingDoc.txt')
for line in file.readlines():
    #for i in range(len(line)):
    listOfTokens = regEx.split(line)
    docArr.append([tok.lower() for tok in listOfTokens if len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장

file = open('data/trainingLabel.txt')
for line in file.readlines():
    curLine = line.strip()
    classArr.append(curLine)

print docArr
print classArr

[['is', 'manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'by', 'trey', 'rose', 'aka', 'dynasty_digest', 'http', 'www', 'dobberbaseball', 'com', 'is', 'manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper', 'machado'], ['machado', 'is', 'off', 'to', 'hot', 'start', 'do', 'you', 'think', 'he', 'll', 'be', 'an', 'allstar', 'mlb', 'orioles', 'http', 'fb', 'me', '5nt5zf7pc'], ['manny', 'machado', 'is', 'freak', 'of', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'that', 'no', 'one', 'else', 'would', 'dream', 'of', 'orioles', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'as', 'an', 'excuse', 'to', 'bask', 'in', 'the', 'glory', 'that', 'machado', 'is', 'our', 'third', 'baseman'], ['machado', 'needs', 'to', 'be', 'signed', 'for', '20', 'years', 'machado', 'orioles'], ['fantasy', 'recommendation', 'by', 'brett', 'talley', 'manny', 'machado', 'http', 'bit', 'ly', '1sycmdh', 'alarm'], ['machado', 'is'

> Code - stop word 생성 및 필터링

In [84]:
stopWord = ['manny', 'machado', 'the', 'www', 'http', 'orioles', 'mlb', 'park', 'maeda', 'kenta', 'dodgers',
            'ladodgers', 'com', 'seunghwanoh', 'seung', 'hwan', 'cardinals', 'stlouis', 'twins', 'debut',
            'hyunsookim', 'kim', 'hyun', 'soo', 'baltimore', 'korean', 'tyler', 'colins', 'tigers', 'tylercollins'
            'keuchel', 'houstonastros', 'giants', 'jepsen', 'braves', 'cubs', 'redSox', 'greinke', 'puig',
            'molina', 'detroitTigers', 'a', 'an', 'for', 'by', 'and', 'of', 'or', 'to', 'is', 'are', 'has',
            'have', 'about', 'i', 'you', 'hyunjinryu', 'hyunjin', 'ryu', 'la', 'byeonghopark', 'byeongho', 
            'park', 'twins', 'minnesota', 'minnesotatwins', 'daeholee', 'daeho', 'lee', 'mariners', 'seattle',
            'seattlemariners','junghokang', 'jungho', 'kang', 'pirates', 'pittsburgh', 'pittsburghpirtes',
            'sinsoochoo', 'sinsoo', 'choo', 'rangers', 'texas', 'texasrangers', 'darvish', 'osaka', 'yuDarvish',
            'rangers', 'he', 'they', 'http', ' ', 'at', 'in', 'him', 'his', 'with', 'that', 'indians', 'royals',
            'SeungHwanOh', 'Cardinals']

def filterDoc(docArr):    
    wordArr = []
    for i in range(len(docArr)):
        temp = []
        for word in docArr[i]:
            word = word.upper().lower()
            if word not in stopWord:
                temp.append(word)

        wordArr.append(temp)
    
    return wordArr

wordArr = filterDoc(docArr)
print wordArr

[['best', 'fantasy', 'baseball', 'player', 'trey', 'rose', 'aka', 'dynasty_digest', 'dobberbaseball', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper'], ['off', 'hot', 'start', 'do', 'think', 'll', 'be', 'allstar', 'fb', 'me', '5nt5zf7pc'], ['freak', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'no', 'one', 'else', 'would', 'dream', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'as', 'excuse', 'bask', 'glory', 'our', 'third', 'baseman'], ['needs', 'be', 'signed', '20', 'years'], ['fantasy', 'recommendation', 'brett', 'talley', 'bit', 'ly', '1sycmdh', 'alarm'], ['just', 'so', 'damn', 'good'], ['moment', 'when', 'your', 'happy', 'your', 'recording', 'game', 'on', 'fire'], ['back', 'back', 'homeruns', 'way', 'go', 'reimold', 'omazing', 'go', 'abc2news', 'ibackthebirds'], ['back', 'back', 'homers', 'let', 'go', 'os', 'homeruns', 'schoop'], ['homers', 'inning', 'reimold', 'scoop', 'mazing'], ['hr', 'giving', 'me', 'boost', 'fantasy', 

## 2.3 분류기 생성
* Naive Bayes 분류기를 이용함
* 232개의 document 중 210(90%)개를 traing set으로, 나머지 22(10%)개의 한국인 메이저리거 관련 document를 test set으로 사용
* 사전확률을 맞추기 위해 training set은 긍정 105개, 부정 105개. test set은 긍정 15개, 부정 7개로 구축.

In [85]:
import bayes
reload(bayes)

myVocabList = bayes.createVocabList(wordArr)

trainMat = []
classMat = []

for postinDoc in wordArr:
    trainMat.append(bayes.setOfWords2Vec(myVocabList, postinDoc))

for strVec in classArr:
    intVec = int(strVec)
    classMat.append(intVec)

p0V,p1V,pAb=bayes.trainNB0(trainMat,classMat)


## 2.4 분석

In [98]:
#regEx = re.compile('\\W*')


#test data 전처리
testDoc = []
testLabel = []
file =open('data/testDoc.txt')

for line in file.readlines():
    lineArr = []
    curLine = line.strip().split('\t')
    
    listOfTokens = regEx.split(curLine[0])
    testDoc.append([tok.lower() for tok in listOfTokens if len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장
    
    testLabel.append(curLine[-1])

filteredDoc = filterDoc(testDoc)

#분류 결과 및 에러율 출력
errorCnt = 0
i = 0
for doc in filteredDoc:
    temp = np.array(bayes.setOfWords2Vec(myVocabList, doc))
    classified = bayes.classifyNB(temp,p0V,p1V,pAb) #Naive bayes 분류기 수행
    
    #에러율 계싼
    if classified != int(testLabel[i]):
        errorCnt += 1
    print 'class is ', testLabel[i], 'classified as ', classified, '\n'
    i += 1
    
print 'Total error rate : ', float(errorCnt)/len(testDoc)    

the word: power is not in my Vocabulary!
the word: pitching is not in my Vocabulary!
the word: 94mph is not in my Vocabulary!
the word: strike is not in my Vocabulary!
class is  1 classified as  0 

the word: reliever is not in my Vocabulary!
the word: koreatimesus is not in my Vocabulary!
the word: relief is not in my Vocabulary!
class is  1 classified as  0 

the word: nickname is not in my Vocabulary!
the word: anything is not in my Vocabulary!
class is  1 classified as  1 

the word: stunning is not in my Vocabulary!
the word: galleryelenash is not in my Vocabulary!
the word: londonartfair is not in my Vocabulary!
the word: impermanence is not in my Vocabulary!
the word: series is not in my Vocabulary!
the word: allmatterwilldecay is not in my Vocabulary!
class is  1 classified as  0 

class is  1 classified as  1 

the word: day is not in my Vocabulary!
the word: day is not in my Vocabulary!
class is  1 classified as  1 

the word: byung is not in my Vocabulary!
the word: entire i

# 3. 결론
* 정확도 약 46% (에러율 : 약 54%)
* 한계
* 1) training data의 수가 부족하여 실제 test data에서는 긍정/부정을 판단불가능한 단어가 많이 발생함
* 2) 트위터 글의 특성상 정서를 나타내는 word가 1~2개에 불과하여 제대로 된 정서 분류가 어려움
* 3) 문법을 지키지 않은 단어 또는 오타가 분류 성능을 떨어뜨리는 요인이 됨
